In this notebook we add carbon biosphere flows to the IPCC 2013 method so that we can have negative CO2 emissions from biomass CCS

import useful libraries and set current project:

In [1]:
%run initialize_notebook.ipynb

# Create IPCC 2013 method will all non-fossil CO2 characterised:

In [2]:
ipcc = Method(('IPCC 2013', 'climate change', 'GWP 100a'))
gwp_data = ipcc.load()

find non-fossil CO2 biosphere flows:

In [3]:
non_fossil = [x for x in get_many(Database("biosphere3"), equals("name", "Carbon dioxide, non-fossil"))]
non_fossil

['Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground'))]

In [4]:
gwp_data.extend([(x.key, 1.) for x in non_fossil])

Find CO2 in air biosphere flow

In [5]:
co2_in_air = get_one(Database("biosphere3"), equals("name", 'Carbon dioxide, in air'))

In [6]:
co2_in_air

'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air'))

In [7]:
gwp_data.append((co2_in_air.key, -1.))

In [8]:
round(pd.Series({get_activity(x)['name']: y for x, y in gwp_data}),2)

Carbon dioxide, fossil                                     1.00
Carbon dioxide, from soil or biomass stock                 1.00
Carbon dioxide, in air                                    -1.00
Carbon dioxide, non-fossil                                 1.00
Carbon dioxide, to soil or biomass stock                  -1.00
Carbon monoxide, fossil                                    4.06
Carbon monoxide, from soil or biomass stock                4.06
Carbon monoxide, non-fossil                                2.49
Chloroform                                                16.40
Dinitrogen monoxide                                      264.80
Ethane, 1,1,1,2-tetrafluoro-, HFC-134a                  1301.27
Ethane, 1,1,1-trichloro-, HCFC-140                       160.10
Ethane, 1,1,1-trifluoro-, HFC-143a                      4804.44
Ethane, 1,1,2-trichloro-1,2,2-trifluoro-, CFC-113       5823.73
Ethane, 1,1-dichloro-1-fluoro-, HCFC-141b                782.04
Ethane, 1,1-difluoro-, HFC-152a         

In [9]:
method = Method(('IPCC 2013', 'climate change', 'GWP 100a', 'Complete'))
method.register()
method.write(gwp_data)
method.process()

# Create a new biosphere flow for non-fossil CO2 to geological storage.

In [10]:
biosphere = Database('biosphere3')
biosphere

Brightway2 SQLiteBackend: biosphere3

In [11]:
new_flow = biosphere.new_activity('CO2 to geological storage, non-fossil', **{'name': 'CO2 to geological storage, non-fossil',
                                                                                'unit': 'kilogram',
                                                                                'type': 'storage',
                                                                                'categories': ('geological storage',)})

new_flow.save()

In [12]:
co2_to_storage = [x for x in Database("biosphere3") if 'CO2 to geological storage, non-fossil' in x['name']][0]

co2_to_storage

'CO2 to geological storage, non-fossil' (kilogram, GLO, ('geological storage',))

Step 2: add characterization for this flow - we will copy a flow for non-fossil CO2 to soil or biomass stock, as it will have the same characterisation factors. 

In [13]:
co2_to_soil = [x for x in Database("biosphere3") if "Carbon dioxide, to soil or biomass stock" in str(x) and "('soil',))" in str(x)][0]
co2_to_soil

'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil',))

Here we have a list of methods that we want to add the flow to:


In [14]:
lcia_methods={
    'CC':('IPCC 2013', 'climate change', 'GWP 100a'),
    'TA':('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'),
    'POF':('ReCiPe Midpoint (H)','photochemical oxidant formation','POFP'),
    'PMF':('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'),
    'MD':('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
    'HT':('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
    'MET':('ReCiPe Midpoint (H)', 'marine ecotoxicity', 'METPinf'),
    'ME':('ReCiPe Midpoint (H)', 'marine eutrophication', 'MEP'),
    'FD':('ReCiPe Midpoint (H)', 'fossil depletion', 'FDP'),
    'IR':('ReCiPe Midpoint (H)', 'ionising radiation', 'IRP_HE'),
    'OD':('ReCiPe Midpoint (H)', 'ozone depletion', 'ODPinf'),
    'FET':('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
    'TET':('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'),
    'ALO':('ReCiPe Midpoint (H)', 'agricultural land occupation', 'ALOP'),
    'NLT':('ReCiPe Midpoint (H)', 'natural land transformation', 'NLTP'),
    'ULO':('ReCiPe Midpoint (H)', 'urban land occupation', 'ULOP'),
    'WD':('ReCiPe Midpoint (H)', 'water depletion', 'WDP'),
    'FE':('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'),
    'R_HH' : ('ReCiPe Endpoint (H,A)', 'human health', 'total'),
    'R_EQ' : ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total'),
    'R_R' : ('ReCiPe Endpoint (H,A)', 'resources', 'total'),
    'R_Total' : ('ReCiPe Endpoint (H,A)', 'total', 'total'),
    
    'CEDB': ('cumulative energy demand','biomass','renewable energy resources, biomass'),
      'CEDF': ('cumulative energy demand','fossil','non-renewable energy resources, fossil'),
      'CEDG': ('cumulative energy demand','geothermal','renewable energy resources, geothermal, converted'),
      'CEDN': ('cumulative energy demand','nuclear','non-renewable energy resources, nuclear'),
      'CEDFr': ('cumulative energy demand','primary forest','non-renewable energy resources, primary forest'),
      'CEDS': ('cumulative energy demand','solar','renewable energy resources, solar, converted'),
      'CEDH': ('cumulative energy demand','water','renewable energy resources, potential (in barrage water), converted'),
      'CEDW': ('cumulative energy demand','wind','renewable energy resources, kinetic (in wind), converted')

    }

In [15]:
for cat in lcia_methods:
    method  = Method(lcia_methods[cat])
    method_data = method.load()
    #first make sure we don't already have the flow included:
    if [x for x in method_data if co2_to_storage.key[1] in x[0][1]]:
        print('Flow already present- you must have run this code already.')
        continue
    
    else:
        try:
            characterized_flow = [x for x in method_data if co2_to_soil.key[1] in x[0][1]][0]
        except:
            continue
        
        method_data.extend([(co2_to_storage.key, characterized_flow[1]) ])
            
        print('Flow added to method: {}'.format(method.name))
        print('Characterisation factor: {}'.format(characterized_flow[1]))
        
        orig_name = [x for x in method.name]
        new_method = Method(tuple(orig_name +['CO2 storage']))
        new_method.register()
        new_method.write(method_data)
        new_method.process()        

Flow added to method: ('IPCC 2013', 'climate change', 'GWP 100a')
Characterisation factor: -1.0
Flow added to method: ('ReCiPe Endpoint (H,A)', 'human health', 'total')
Characterisation factor: -0.277227722772
Flow added to method: ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total')
Characterisation factor: -0.175224928177
Flow added to method: ('ReCiPe Endpoint (H,A)', 'total', 'total')
Characterisation factor: -0.452452650949


In [16]:
new_methods = [x for x in methods if 'CO2 storage' in str(x)]
new_methods

[('IPCC 2013', 'climate change', 'GWP 100a', 'CO2 storage'),
 ('ReCiPe Endpoint (H,A)', 'human health', 'total', 'CO2 storage'),
 ('ReCiPe Endpoint (H,A)', 'ecosystem quality', 'total', 'CO2 storage'),
 ('ReCiPe Endpoint (H,A)', 'total', 'total', 'CO2 storage')]

In [18]:
Method(new_methods[0]).load()[10:]

[(('biosphere3', 'e8787b5e-d927-446d-81a9-f56977bbfeb4'), 1.0),
 (('biosphere3', '259cf8d6-6ea8-4ccf-84b7-23c930a5b2b3'), -1.0),
 (('biosphere3', '8ae4d8bb-3e4b-4825-8325-94696d7a64fd'), -1.0),
 (('biosphere3', '60d424f7-d5a9-4549-9540-da06684bc3bb'), -1.0),
 (('biosphere3', '375bc95e-6596-4aa1-9716-80ff51b9da77'), -1.0),
 (('biosphere3', '49c594c6-b6a8-42a3-95c5-cca812fda80b'), 4.0624),
 (('biosphere3', 'a3beb5ac-5149-47f6-a035-53fc5030f10a'), 4.0624),
 (('biosphere3', '099b36ab-4c03-4587-87f4-2f81e337afb8'), 4.0624),
 (('biosphere3', 'ba2f3f82-c93a-47a5-822a-37ec97495275'), 4.0624),
 (('biosphere3', '6edcc2df-88a3-48e1-83d8-ffc38d31c35b'), 4.0624),
 (('biosphere3', '05f6e226-0593-46f5-a788-01fa306e66ad'), 4.0624),
 (('biosphere3', 'bd086aa3-312a-47af-a7b2-04fddbf8f01e'), 4.0624),
 (('biosphere3', 'e1da7e11-cc2c-4ea2-a42a-bf558150014b'), 4.0624),
 (('biosphere3', '68b9b577-90cf-49e4-88bb-a55d80f3ac5d'), 4.0624),
 (('biosphere3', '69a6c884-39be-444f-a67c-7436a5e66de2'), 4.0624),
 (('bi